In [10]:
# Test.
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import caffe
import sys
import time

imagenet_root = '/home/haow3/occlusion-project/data/imagenet/'
caffe_root = '/home/haow3/software/caffe-rc3/'
new_to_original_class_id = [99, 113, 132, 144, 321, 344, 386, 494, 497, 512, 525, 526, \
650, 656, 737, 763, 844, 848, 903, 945]
original_to_new_class_id = {512: 9, 321: 4, 386: 6, 99: 0, 132: 2, 848: 17, 497: 8,\
650: 12, 903: 18, 844: 16, 525: 10, 494: 7, 144: 3, 113: 1, 763: 15, 526: 11, 344: 5,\
656: 13, 737: 14, 945: 19}

plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

caffe.set_device(1)
caffe.set_mode_gpu()

is_finetuned = True
net_name = 'all'
#net = caffe.Net(imagenet_root + 'model/bvlc_alexnet/deploy.prototxt',
#                imagenet_root + 'model/bvlc_alexnet/bvlc_alexnet.caffemodel',
#                caffe.TEST)
net = caffe.Net(imagenet_root + 'model/finetune_alexnet_{}/deploy.prototxt'.format(net_name),
                imagenet_root + 'model/finetune_alexnet_{}/finetune_alexnet_{}.caffemodel'.format(net_name, net_name),
                caffe.TEST)

# input preprocessing: 'data' is the name of the input blob == net.inputs[0]
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
transformer.set_transpose('data', (2,0,1))
transformer.set_mean('data', np.load(caffe_root + 'python/caffe/imagenet/ilsvrc_2012_mean.npy').mean(1).mean(1)) # mean pixel
transformer.set_raw_scale('data', 255)  # the reference model operates on images in [0,255] range instead of [0,1]
transformer.set_channel_swap('data', (2,1,0))  # the reference model has channels in BGR order instead of RGB

# set net to batch size of 50
net.blobs['data'].reshape(50,3,227,227)

test_dataset = [(0.0, 0), (1.0/3, 3), (1.0/2, 3), (2.0/3, 2), (1.0, 1)]
positive_true = [0 for i in test_dataset]
image_sum = [0 for i in test_dataset]
accuracy = [0.0 for i in test_dataset]

start_time = time.time()
for index, (occlu_size, occlu_num) in enumerate(test_dataset):
    percent = str(int(100 * occlu_size * occlu_size))
    print percent, occlu_num
    test_file = open('{}dataset/test_{}_{}.txt'.format(imagenet_root, percent, str(occlu_num)), 'r')
    
    lines = test_file.readlines()
    image_sum[index] = len(lines)
    for i, line in enumerate(lines):
        # A possible optimization to consider: add occluders on the test image before feeding them.
        image_path, class_id = line.split(' ')
        class_id = int(class_id)
        net.blobs['data'].data[...] = transformer.preprocess('data', caffe.io.load_image(image_path))
        out = net.forward()
        predict_id = out['prob'][0].argmax()
        if is_finetuned:
            true_id = class_id
        else:
            true_id = new_to_original_class_id[class_id]
        now_time = time.strftime("%H:%M:%S", time.gmtime(int(time.time() - start_time)))
        if i % 100 == 0:
            print '[{}] {:5}/{:5}: {} {} {}'.format(now_time, i, image_sum[index], image_path, predict_id, true_id)
        if true_id == predict_id:
            positive_true[index] += 1        
    accuracy[index] = positive_true[index] / float(image_sum[index])
    
    print 'positive_true: ', positive_true
    print 'image_sum: ', image_sum
    print 'accuracy: ', accuracy
    

0 0
[00:00:00]     0/ 2000: /home/haow3/occlusion-project/data/imagenet/dataset/test_0_0/n01855672_13294_0.jpeg 0 0
[00:00:04]   100/ 2000: /home/haow3/occlusion-project/data/imagenet/dataset/test_0_0/n02009912_4182_0.jpeg 2 2
[00:00:09]   200/ 2000: /home/haow3/occlusion-project/data/imagenet/dataset/test_0_0/n02051845_331_0.jpeg 3 3
[00:00:13]   300/ 2000: /home/haow3/occlusion-project/data/imagenet/dataset/test_0_0/n02398521_935_0.jpeg 5 5
[00:00:17]   400/ 2000: /home/haow3/occlusion-project/data/imagenet/dataset/test_0_0/n03017168_2788_0.jpeg 17 7
[00:00:21]   500/ 2000: /home/haow3/occlusion-project/data/imagenet/dataset/test_0_0/n03028079_14885_0.jpeg 8 8
[00:00:26]   600/ 2000: /home/haow3/occlusion-project/data/imagenet/dataset/test_0_0/n03109150_26169_0.jpeg 9 9
[00:00:30]   700/ 2000: /home/haow3/occlusion-project/data/imagenet/dataset/test_0_0/n03160309_11747_0.jpeg 10 10
[00:00:34]   800/ 2000: /home/haow3/occlusion-project/data/imagenet/dataset/test_0_0/n03179701_22241_0.

In [11]:
print 'positive_true =', positive_true
print 'image_sum =' , image_sum
print 'accuracy =', accuracy

positive_true = [1867, 16463, 15908, 6637, 892]
image_sum = [2000, 18000, 18000, 8000, 2000]
accuracy = [0.9335, 0.9146111111111112, 0.8837777777777778, 0.829625, 0.446]
